# imports

In [1]:
#!pip install --upgrade jinja2==3.0.1
#!pip install pixiedust
#!pip install torch_stft

#import pixiedust


In [4]:
from __future__ import print_function
import argparse
import pickle

import numpy as np
import torch
import torch.optim as optim
from gcommand_loader import GCommandLoader
from train import train, test
from attacks import attack

import os
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [5]:
class args():

    def _init_(self, name):
        pass
    def cuda(self,x):
        pass         
    def train_path(self,x):
        pass
    def test_path(self,x):
        pass
    def test_batch_size(self,x):
        pass
    def arc(self,x):
        pass
    def epochs(self,x):
        pass
    def lr(self,x):
        pass
    def momentum(self,x):
        pass
    def optimizer(self,x):
        pass    
    def log_interval(self,x):
        pass
    def seed(self,x):
        pass
    def patience(self,x):
        pass
    def window_size(self,x):
        pass
    def window_stride(self,x):
        pass
    def window_type(self,x):
        pass    
    def normalize(self,x):
        pass
    def chkpt_path(self,x):
        pass
    def chkpt_path_train(self,x):
        pass
    def n_iter(self,x):
        pass
    def eps(self,x):
        pass    
    def alpha(self,x):
        pass
    def test_mode(self,x):
        pass

In [6]:
#args.train_path='/home/maayanl/data/gcommands/train'#
#args.test_path='/home/maayanl/data/gcommands/test'
#args.valid_path='/home/maayanl/data/gcommands/valid'
args.train_path='/home/maayanl/data/data2/train'#
args.test_path='/home/maayanl/data/data2/test'
args.valid_path='/home/maayanl/data/data2/valid'
args.batch_size=50
args.test_batch_size=50
args.arc='LeNet'#'network architecture: LeNet, VGG11, VGG13, VGG16, VGG19'
args.epochs=20#TODO:default=100
args.lr=0.001
args.momentum=0.9#help='SGD momentum, for SGD only'
args.optimizer='adam'#' help=optimization method: sgd | adam'
args.cuda=True
args.log_interval=50
args.cuda = args.cuda and torch.cuda.is_available()
args.seed="1234"
args.patience=1
args.window_size=0.2
args.window_stride=0.1
args.window_type='hamming'
args.normalize=True

args.test_mode=False #parser.add_argument('--test_mode', action="store_true", help='Whether to run model for test only or not')
args.chkpt_path="chkpt_path"
args.chkpt_path_train="chkpt_path_train"
args.n_iter=10
args.eps=0.01
args.alpha=0.01

In [7]:
import torch.nn as nn
import torch.nn.functional as F
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, kernel_size=5)
        self.conv2 = nn.Conv2d(20, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(174680, 1000)
        self.fc2 = nn.Linear(1000, 2)#TODO:WAS 35 run1

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x)

In [8]:
args.cuda = args.cuda and torch.cuda.is_available()
torch.manual_seed(args.seed)
if args.cuda:
    torch.cuda.manual_seed(args.seed)

# build model
if args.arc == 'LeNet':
    model = LeNet()
elif args.arc.startswith('VGG'):
    model = VGG(args.arc)
else:
    model = LeNet()

if args.cuda:
    model = model.cuda()

# loading data
train_dataset = GCommandLoader(args.train_path, window_size=args.window_size, window_stride=args.window_stride,
                             window_type=args.window_type, normalize=args.normalize)
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=args.batch_size, shuffle=True,
    num_workers=8, pin_memory=args.cuda, sampler=None)

valid_dataset = GCommandLoader(args.valid_path, window_size=args.window_size, window_stride=args.window_stride,
                             window_type=args.window_type, normalize=args.normalize)
valid_loader = torch.utils.data.DataLoader(
    valid_dataset, batch_size=args.batch_size, shuffle=None,
    num_workers=8, pin_memory=args.cuda, sampler=None)

test_dataset = GCommandLoader(args.test_path, window_size=args.window_size, window_stride=args.window_stride,
                            window_type=args.window_type, normalize=True)
test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=args.test_batch_size, shuffle=None,
    num_workers=8, pin_memory=args.cuda, sampler=None)



# define optimizer
if args.optimizer.lower() == 'adam':
    optimizer = optim.Adam(model.parameters(), lr=args.lr)
elif args.optimizer.lower() == 'sgd':
    optimizer = optim.SGD(model.parameters(), lr=args.lr,
                          momentum=args.momentum)
else:
    optimizer = optim.SGD(model.parameters(), lr=args.lr,
                          momentum=args.momentum)

best_valid_loss = np.inf
iteration = 0
epoch = 1



In [10]:
if args.test_mode:
    checkpoint = torch.load(args.chkpt_path)
    model.load_state_dict(checkpoint['net'].state_dict())
    model.eval()

# train int with early stopping
while (epoch < args.epochs + 1) and (iteration < args.patience) and not args.test_mode:
    train(train_loader, model, optimizer, epoch, args.cuda, args.log_interval)
    valid_loss = test(valid_loader, model, args.cuda)
    if valid_loss > best_valid_loss:
        if epoch > 11:
           for param_group in optimizer.param_groups:
              param_group['lr'] = args.lr*0.2
        iteration += 1
        print('Loss was not improved, iteration {0}'.format(str(iteration)))
    else:
        print('Saving model...')
        iteration = 0
        best_valid_loss = valid_loss
        state = {
            'net': model,
            'acc': valid_loss,
            'epoch': epoch,
            'optimizer_state_dict:': optimizer.state_dict()
        }
        if not os.path.isdir('checkpoint'):
            os.mkdir('checkpoint')
        torch.save(state, f'./checkpoint/{args.chkpt_path_train}.t7')
    epoch += 1


test(test_loader, model, args.cuda)
#attack(test_loader, 0, model, n_iter=10,eps=args.eps, alpha =args.alpha, rand_init=True)

/tmp/ipykernel_427/2616342028.py:19: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


Train Epoch: 1 [0/3291 (0%)]	Loss: 0.700258
Train Epoch: 1 [2500/3291 (76%)]	Loss: 0.617960


/usr/local/lib/python3.8/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Average loss: 0.4085, Accuracy: 318/395 (81%)

Saving model...
Train Epoch: 2 [0/3291 (0%)]	Loss: 0.474551
Train Epoch: 2 [2500/3291 (76%)]	Loss: 0.420505

Test set: Average loss: 0.4274, Accuracy: 362/395 (92%)

Loss was not improved, iteration 1

Test set: Average loss: 0.4573, Accuracy: 346/392 (88%)



0.4573273415468177

In [12]:
import os

import torch
import librosa
import numpy as np
import torch.nn.functional as F

def snr_db(x, desired_noise):
    return 20 * torch.log10(torch.sum(x ** 2) / torch.sum(desired_noise ** 2))

def relative_eps(snr):
    return torch.pow(torch.tensor(10.0), float(snr) / 20)

def pad_audio(audio, orig_size):
    max_size = (16000,)

    if audio.shape[1] < max_size[0]:
        torch.nn.functional.pad(audio,(0,max_size[0] - orig_size[1]))
    elif audio.shape[1] > max_size[0]:
        audio = audio[:max_size[0]]
    return audio

def attack_wav(dataloader, x,y, model, n_iter, epsilon, alpha, rand_init_uniform=True,rand_init_normal=False, clip_min_max=True):
    model.eval()
    correct = 0
    total = 0

    y = torch.LongTensor([y]).cuda()
    audio, sr = librosa.load(x, sr=None)
    min_val, max_val = -1, 1 # audio.min(), audio.max()
    audio = np.expand_dims(audio, 0)

    audio = torch.FloatTensor(audio)
    delta = torch.zeros_like(audio, requires_grad=True)

    if rand_init_normal:
        delta.data = delta.data.normal_(std=1)
    elif rand_init_uniform:
        delta.data = delta.data.uniform_(-epsilon, epsilon)
    
    padded_audio = pad_audio(audio + delta,audio.shape)

    
    for i in range(n_iter):
        spect, phase = dataloader.dataset.stft.transform(padded_audio)

        spect = spect.unsqueeze(0).cuda()
        yhat = model(spect)
        loss = F.nll_loss(yhat, y)
        loss.backward()
        
        delta.data = delta.data + alpha * torch.sign(delta.grad.data)
        delta.data = torch.clamp(delta.data, min=-epsilon, max=epsilon)
        if clip_min_max:
            delta.data = torch.clamp(audio+delta.data, min=-min_val, max=max_val)-audio
        
    total += 1
    correct += int(yhat.argmax() != y.item())

    print(f"PGD accuracy {correct*100/total} ({correct}/{total})")
    return audio+delta.detach(), sr, correct


def attack(dataloader, example_idx, model, n_iter=10, eps=0.01, alpha=0.01/2, rand_init=True):
    x, y = dataloader.dataset.spects[example_idx]
    adv_wav, sr, cor = attack_wav(dataloader, x, y, model, n_iter, eps, alpha, rand_init)
    return adv_wav




In [15]:
test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=1, shuffle=None,
    num_workers=8, pin_memory=args.cuda, sampler=None)
attack(test_loader, 0, model, n_iter=10,eps=args.eps, alpha =args.alpha, rand_init=True)

RuntimeError: Calculated padded input size per channel: (798 x 3). Kernel size: (5 x 5). Kernel size can't be greater than actual input size